In [1]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Mounted at /gdrive
/gdrive


In [2]:
pip install minisom


  Created wheel for minisom: filename=MiniSom-2.2.9-py3-none-any.whl size=8594 sha256=36c9084f435fd2191effa3a6a760560c36c63603220eb0c7ec7dab84e867d820
  Stored in directory: /root/.cache/pip/wheels/3d/a1/10/f50b6f4865652eac239a2700de411c3078c27e1318320e494c
Successfully built minisom


In [3]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
# Calculate silhouette_score
from sklearn.metrics import silhouette_score
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")


In [4]:
#Read files
trainfile = r'/gdrive/My Drive/CIS 508/Assignment 4/Problem 1/CC GENERAL.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe

print(trainData.shape)


(8950, 18)


##Exploratory Data Analysis

In [16]:
trainData.isna().sum()

CUST_ID                               0
BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64

In [5]:
#fills in null values with most common value in column
X_train = trainData.apply(lambda x:x.fillna(x.value_counts().index[0]))
cols_to_drop = 'CUST_ID'
X_train = X_train.drop([cols_to_drop], axis=1)
#scales all features in dataframe 
x = X_train.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_train = pd.DataFrame(x_scaled,columns=X_train.columns)


In [18]:
X_train

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,0.002148,0.818182,0.001945,0.000000,0.004240,0.000000,0.166667,0.000000,0.083333,0.000000,0.00000,0.005587,0.031720,0.003979,0.001826,0.000000,1.0
1,0.168169,0.909091,0.000000,0.000000,0.000000,0.136685,0.000000,0.000000,0.000000,0.166667,0.03252,0.000000,0.232053,0.080893,0.014034,0.222222,1.0
2,0.131026,1.000000,0.015766,0.018968,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.033520,0.248748,0.012264,0.008210,0.000000,1.0
3,0.087521,0.636364,0.030567,0.036775,0.000000,0.004366,0.083333,0.083333,0.000000,0.055555,0.00813,0.002793,0.248748,0.000000,0.003918,0.000000,1.0
4,0.042940,1.000000,0.000326,0.000393,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.00000,0.002793,0.038397,0.013374,0.003204,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8945,0.001496,1.000000,0.005936,0.000000,0.012939,0.000000,1.000000,0.000000,0.833333,0.000000,0.00000,0.016760,0.031720,0.006419,0.000640,0.500000,0.0
8946,0.001007,1.000000,0.006118,0.000000,0.013333,0.000000,1.000000,0.000000,0.833333,0.000000,0.00000,0.016760,0.031720,0.005439,0.003918,0.000000,0.0
8947,0.001229,0.833333,0.002945,0.000000,0.006418,0.000000,0.833333,0.000000,0.666667,0.000000,0.00000,0.013966,0.031720,0.001602,0.001078,0.250000,0.0
8948,0.000707,0.833333,0.000000,0.000000,0.000000,0.000776,0.000000,0.000000,0.000000,0.111111,0.01626,0.000000,0.015025,0.001036,0.000729,0.250000,0.0


In [19]:
X_train.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000
mean,0.082154,0.877271,0.020457,0.014534,0.018270,0.020766,0.490351,0.202458,0.364437,0.090096,0.026413,0.041089,0.148390,0.034170,0.011052,0.153715,0.919553
std,0.109306,0.236904,0.043570,0.040722,0.040193,0.044491,0.401371,0.298336,0.397448,0.133414,0.055485,0.069435,0.121491,0.057078,0.030533,0.292499,0.223055
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.006736,0.888889,0.000808,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.002793,0.051753,0.007556,0.002236,0.000000,1.000000
50%,0.045864,1.000000,0.007367,0.000932,0.003956,0.000000,0.500000,0.083333,0.166667,0.000000,0.000000,0.019553,0.098497,0.016894,0.003918,0.000000,1.000000
75%,0.107868,1.000000,0.022637,0.014166,0.020828,0.023629,0.916667,0.300000,0.750000,0.148148,0.032520,0.047486,0.215359,0.037482,0.010322,0.142857,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


##Purchases_Frequency Clustering

In [25]:
#K-Means Clustering ========================================================================
#Default Mode - K=8
pfCluster=pd.DataFrame(X_train['PURCHASES_FREQUENCY'])
kmeans = KMeans()
kmeans.fit(pfCluster)
print(kmeans.labels_)
print(kmeans.cluster_centers_)
#Print Silhouette measure
print(silhouette_score(pfCluster, kmeans.labels_))
print(pfCluster.shape)


[4 0 5 ... 1 0 3]
[[1.60427227e-14]
 [8.75487270e-01]
 [3.72574621e-01]
 [7.05885772e-01]
 [2.07677856e-01]
 [1.00000000e+00]
 [5.39051090e-01]
 [8.83837278e-02]]
0.8194937479589229
(8950, 1)


In [26]:
#Add Cluster Number to each datapoint + save file
df_kmeans=pfCluster.copy()
kmeans_predict_train=kmeans.predict(pfCluster)
print(kmeans_predict_train)

df_kmeans['Cluster Number']=kmeans.labels_
print(pfCluster.shape)
print(df_kmeans.shape)
print(df_kmeans.head())
df_kmeans.dtypes

export_csv = df_kmeans.to_csv(r'/gdrive/My Drive/CIS 508/Assignment 4/Problem 1/PF_Train-8.csv')


[4 0 5 ... 1 0 3]
(8950, 1)
(8950, 2)
   PURCHASES_FREQUENCY  Cluster Number
0             0.166667               4
1             0.000000               0
2             1.000000               5
3             0.083333               7
4             0.083333               7


###Hyperparameter Tuning

In [28]:
#K-Means Clustering ========================================================================
# K=20
#X_train = trainData.drop(["TARGET"], axis=1) #extracting training data without the target column

pfCluster2=pd.DataFrame(X_train['PURCHASES_FREQUENCY'])
kmeans = KMeans(n_clusters=20, init='random', algorithm='elkan')
kmeans.fit(pfCluster2)
kmeans_predict_train=kmeans.predict(pfCluster2)
print(kmeans_predict_train)

#print(kmeans.labels_)
print(kmeans.cluster_centers_)
print(pfCluster2.shape)
#Print Silhouette measure
print(silhouette_score(pfCluster2, kmeans.labels_))

[ 3  5  6 ... 10  5 16]
[[3.32192661e-01]
 [8.66066826e-02]
 [2.08602065e-01]
 [1.65799157e-01]
 [2.51928831e-01]
 [1.60427227e-14]
 [1.00000000e+00]
 [6.32727520e-01]
 [8.66246569e-01]
 [7.50546452e-01]
 [8.31799084e-01]
 [5.80740768e-01]
 [3.69318000e-01]
 [7.15251615e-01]
 [9.16166698e-01]
 [4.52440625e-01]
 [6.66667000e-01]
 [8.95238143e-01]
 [4.16527368e-01]
 [5.00000000e-01]]
(8950, 1)
0.9593884583988488


In [29]:
#Add Cluster Number to each datapoint + save file
df_kmeans=pfCluster2.copy()
kmeans_predict_train=kmeans.predict(pfCluster2)
print(kmeans_predict_train)

df_kmeans['Cluster Number']=kmeans_predict_train
print(df_kmeans.shape)
print(df_kmeans.head())
print(df_kmeans.dtypes)

export_csv = df_kmeans.to_csv(r'/gdrive/My Drive/CIS 508/Assignment 4/Problem 1/PF_Train-20.csv')


[ 3  5  6 ... 10  5 16]
(8950, 2)
   PURCHASES_FREQUENCY  Cluster Number
0             0.166667               3
1             0.000000               5
2             1.000000               6
3             0.083333               1
4             0.083333               1
PURCHASES_FREQUENCY    float64
Cluster Number           int32
dtype: object


##Balance Clustering

In [30]:
#K-Means Clustering ========================================================================
#Default Mode - K=8
bCluster=pd.DataFrame(X_train['BALANCE'])
kmeans = KMeans()
kmeans.fit(bCluster)
print(kmeans.labels_)
print(kmeans.cluster_centers_)
#Print Silhouette measure
print(silhouette_score(bCluster, kmeans.labels_))
print(bCluster.shape)


[3 6 6 ... 3 3 3]
[[0.44463851]
 [0.04992248]
 [0.2241861 ]
 [0.00790329]
 [0.31531816]
 [0.67667104]
 [0.14683923]
 [0.0898028 ]]
0.6306994479342294
(8950, 1)


In [31]:
#Add Cluster Number to each datapoint + save file
df_kmeans=bCluster.copy()
kmeans_predict_train=kmeans.predict(bCluster)
print(kmeans_predict_train)

df_kmeans['Cluster Number']=kmeans.labels_
print(bCluster.shape)
print(df_kmeans.shape)
print(df_kmeans.head())
df_kmeans.dtypes

export_csv = df_kmeans.to_csv(r'/gdrive/My Drive/CIS 508/Assignment 4/Problem 1/B_Train-8.csv')


[3 6 6 ... 3 3 3]
(8950, 1)
(8950, 2)
    BALANCE  Cluster Number
0  0.002148               3
1  0.168169               6
2  0.131026               6
3  0.087521               7
4  0.042940               1


###Hyperparameter Tuning

In [32]:
#Add Cluster Number to each datapoint + save file
#BALANCE

bCluster2=pd.DataFrame(X_train['BALANCE'])
kmeans = KMeans(n_clusters=15, init='random', algorithm='elkan')
kmeans.fit(bCluster2)
kmeans_predict_train=kmeans.predict(bCluster2)
print(kmeans_predict_train)
#print(kmeans.labels_)
print(kmeans.cluster_centers_)
print(bCluster2.shape)
#Print Silhouette measure
print(silhouette_score(bCluster2, kmeans.labels_))



[9 7 8 ... 9 9 5]
[[0.04122461]
 [0.30391144]
 [0.73501292]
 [0.01181349]
 [0.19534318]
 [0.02463456]
 [0.24392681]
 [0.15458906]
 [0.12462669]
 [0.00230632]
 [0.05737599]
 [0.39003907]
 [0.0973713 ]
 [0.07593465]
 [0.51180825]]
(8950, 1)
0.5763605781987302


In [33]:
#Add Cluster Number to each datapoint + save file
df_kmeans=bCluster2.copy()
kmeans_predict_train=kmeans.predict(bCluster2)
print(kmeans_predict_train)

df_kmeans['Cluster Number']=kmeans.labels_
print(bCluster2.shape)
print(df_kmeans.shape)
print(df_kmeans.head())
df_kmeans.dtypes

export_csv = df_kmeans.to_csv(r'/gdrive/My Drive/CIS 508/Assignment 4/Problem 1/B_Train-15.csv')


[9 7 8 ... 9 9 5]
(8950, 1)
(8950, 2)
    BALANCE  Cluster Number
0  0.002148               9
1  0.168169               7
2  0.131026               8
3  0.087521              12
4  0.042940               0
